For scraping data from the Global Footprint Network.
https://www.footprintnetwork.org/

Countrycode Canada: 33

data types: 
biocapacity per capita (gha / person): 'BCpc'
biocapacity (gha): 'BCtot'
ecological footprint per capita (gha / person): 'EFCpc'
ecological footprint (gha): 'EFCtot'
number of earths: 'earth'


In [19]:
import pandas as pd
import requests
import json

In [20]:
def download(country, datastring): 
    
    countrycode = {'Canada':33,
                    'World':5001}[country]
    headers = {
        'Accept': 'application/json',
    }
    key = '13CMLmdbBei8dBCKf4tv3Gq7u2Vkg89so04K2reij4tm0l8o17r6'
    url_base = 'https://api.footprintnetwork.org/v1/data/'

    yearout = []
    valueout = []
    years = list(range(1961,2017))

    for year in years:
        url = url_base+str(countrycode)+'/'+str(year)+'/'+datastring
        r = requests.get(url, headers=headers, auth=('any-user-name', key))
        yearout.append((json.loads(r.content))[0]['year'])
        valueout.append((json.loads(r.content))[0]['value'])

    df = pd.DataFrame({'year':yearout, datastring+'_'+country:valueout})
    return df 


In [21]:
def mergedata(country):
  
    df = pd.DataFrame({'year':list(range(1961,2017))})
    
    for datastring in ['BCpc','BCtot','EFCpc','EFCtot','earth']:
        dg = download(country,datastring)
        df = df.merge(dg, on='year', how='left')
        
    return df

In [22]:
def mergecountries():
    
    df = pd.DataFrame({'year':list(range(1961,2017))})
    
    for country in ['Canada', 'World']:
        dg = mergedata(country)
        df = df.merge(dg, on = 'year', how='left')
        
    return df


In [23]:
df = mergecountries()

In [81]:
df.to_csv('~/code/footprint/static/footprintnetwork.csv')
#df2=pd.read_csv('~/code/footprint/static/footprintnetwork.csv')

In [79]:
import datetime as dt
leaps = [1964,1968,1972,1976,1980,1984,1988,1992,1996,2000,2004,2008,2012,2016] 

for i in df['year']:
    if i in leaps:
        df['od_world'] = (df['BCtot_World'] / df['EFCtot_World']) * 366
        df['od_can_pc'] = (df['BCpc_World']/ df['EFCpc_Canada']) * 366
    else:
        df['od_world'] = (df['BCtot_World'] / df['EFCtot_World']) * 365
        df['od_can_pc'] = (df['BCpc_World']/ df['EFCpc_Canada']) * 365
    

df['odate_world'] = [pd.to_datetime('1/1/1900') + dt.timedelta(days=i) for i in df['od_world']]
df['odate_can'] = [pd.to_datetime('1/1/1900') + dt.timedelta(days=i) for i in df['od_can_pc']]



In [80]:
df[df['year']==2016]['odate_world']

55   1900-08-06 04:05:34.998163
Name: odate_world, dtype: datetime64[ns]

In [67]:
df2[df2['year']==2016]['odate_world']

55    1900-08-05 13:51:08.509097
Name: odate_world, dtype: object

In [54]:
df.head()

,year,BCpc_Canada,BCtot_Canada,EFCpc_Canada,EFCtot_Canada,earth_Canada,BCpc_World,BCtot_World,EFCpc_World,EFCtot_World,earth_World,od_world,od_can_pc,odate_world,odate_can
0,1961,26.269294,4.808437e+08,7.386358,1.352029e+08,2.370565,3.115864,9.628986e+09,2.281760,7.051336e+09,0.732304,498.427475,153.971732,1901-05-14 10:15:33.877515,1900-06-03 23:19:17.642986
1,1962,26.616808,4.968985e+08,8.821525,1.646855e+08,2.867392,3.076497,9.688629e+09,2.313300,7.285146e+09,0.751926,485.419195,127.293360,1901-05-01 10:03:38.409020,1900-05-08 07:02:26.326283
2,1963,26.570556,5.052843e+08,9.321593,1.772659e+08,3.089634,3.017054,9.685557e+09,2.357557,7.568398e+09,0.781410,467.103901,118.136954,1901-04-13 02:29:37.032621,1900-04-29 03:17:12.856434
3,1964,25.589469,4.953916e+08,8.738150,1.691636e+08,2.938162,2.974019,9.735957e+09,2.408931,7.886046e+09,0.809992,450.621813,124.227319,1901-03-27 14:55:24.654735,1900-05-05 05:27:20.404508
4,1965,25.508971,5.026262e+08,9.266734,1.825908e+08,3.171051,2.922291,9.759252e+09,2.448154,8.175837e+09,0.837752,435.689574,115.103776,1901-03-12 16:32:59.212402,1900-04-26 02:29:26.248989


In [82]:
max(df['earth_World'])


1.7032334153714166

In [205]:
# plotly implementation
import plotly
import plotly.graph_objects as go
import plotly.figure_factory as ff

fig = go.Figure(
        data = [
            go.Bar(
                x=df['year'],
                y=df['od_can_pc']
                  ),
            go.Bar(
                x=df['year'],
                y=df['od_world']
                )
              ]
    )
#fig.update_yaxes(autorange="reversed")


fig.show()

In [2]:
import pandas as pd
import json

df = pd.read_csv("~/code/footprint/static/footprintnetwork.csv")
chart_data = json.dumps(df.to_dict(orient='records'), indent=2)
print(chart_data)

[
  {
    "Unnamed: 0": 0,
    "year": 1961,
    "BCpc_Canada": 26.26929358,
    "BCtot_Canada": 480843667.7,
    "EFCpc_Canada": 7.3863582789999995,
    "EFCtot_Canada": 135202859.4,
    "earth_Canada": 2.3705650080000003,
    "BCpc_World": 3.115864047,
    "BCtot_World": 9628985823,
    "EFCpc_World": 2.281760499,
    "EFCtot_World": 7051336451,
    "earth_World": 0.732304255,
    "od_world": 498,
    "od_can_pc": 153.971732,
    "odate_world": "15:33.9",
    "odate_can": "19:17.6"
  },
  {
    "Unnamed: 0": 1,
    "year": 1962,
    "BCpc_Canada": 26.61680823,
    "BCtot_Canada": 496898535.7,
    "EFCpc_Canada": 8.821524945,
    "EFCtot_Canada": 164685517.1,
    "earth_Canada": 2.867392292,
    "BCpc_World": 3.076497405,
    "BCtot_World": 9688628688,
    "EFCpc_World": 2.3132996180000003,
    "EFCtot_World": 7285145521,
    "earth_World": 0.7519264,
    "od_world": 485,
    "od_can_pc": 127.29336029999999,
    "odate_world": "03:38.4",
    "odate_can": "02:26.3"
  },
  {
    "Unname

In [7]:
months = 
['Jan','Feb','Mar','Apr','May','Jun',
 'Jul','Aug','Sep','Oct','Nov','Dec']

values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
df = pd.DataFrame({'month':months, 'value':values})

In [17]:
df

,month,value
0,Jan,1
1,Feb,2
2,Mar,3
3,Apr,4
4,May,5
5,Jun,6
6,Jul,7
7,Aug,8
8,Sep,9
9,Oct,10
